In [1]:
import pickle
import pandas as pd
import numpy as np
from scipy.spatial import distance
import scipy.stats as stats
import csv

In [2]:
from collections import defaultdict, Counter

In [3]:
# instantiate a dict phone --> feature vector
with open('data/phoneme_vectors/feature_dict.p', "rb") as input_file:
    phone2vec = pickle.load(input_file)

In [4]:
word2ipa = {}

with open('data/words_ipa/de.tsv') as csv_file:
    
    csv_reader = csv.DictReader(csv_file, delimiter='\t')#(csv_file, delimiter='\t', )
    
    
    #print(csv_reader)
    
    for row in csv_reader:
        word2ipa[row["Word"]] =  row["IPA transcription"]
        

In [5]:
word2ipa['Lösung']

'l øː z ʊ ŋ'

In [6]:
# phoneme set
PHONEME_SET = set([ipa  for w, ipa in set(word2ipa.items()) for ipa in ipa.split() ])

In [7]:
PHONEME_SET

{'a',
 'aː',
 'b',
 'd',
 'eː',
 'f',
 'h',
 'i',
 'iː',
 'j',
 'k',
 'l',
 'm',
 'n',
 'oː',
 'p',
 'r',
 's',
 't',
 'uː',
 'v',
 'x',
 'yː',
 'z',
 'ç',
 'øː',
 'ŋ',
 'œ',
 'ɐ',
 'ɔ',
 'ə',
 'ɛ',
 'ɛː',
 'ɡ',
 'ɪ',
 'ʀ',
 'ʃ',
 'ʊ',
 'ʏ',
 'ʒ'}

In [8]:
phone2dist = defaultdict(lambda: defaultdict(float))

for p1 in PHONEME_SET:
    for p2 in PHONEME_SET:
        
        dist = distance.hamming(phone2vec[p1], phone2vec[p2])#**(1/2)

        phone2dist[p1][p2] = dist

In [9]:
phone2dist['z']['s'], phone2dist['p']['b'], phone2dist['p']['z'], phone2dist['p']['ʊ'],  

(0.02631578947368421, 0.02631578947368421, 0.2631578947368421, 0.5)

In [10]:
# String-based Levenstein Distance (PWLD)
def LD(s, t):
        ''' From Wikipedia article; Iterative with two matrix rows. '''
        if s == t: return 0
        elif len(s) == 0: return len(t)
        elif len(t) == 0: return len(s)
        v0 = [None] * (len(t) + 1)
        v1 = [None] * (len(t) + 1)
        for i in range(len(v0)):
            v0[i] = i
        for i in range(len(s)):
            v1[0] = i + 1
            for j in range(len(t)):
                cost = 0 if s[i] == t[j] else 1
                v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
            for j in range(len(v0)):
                v0[j] = v1[j]
                
        return v1[len(t)]

In [100]:
# Phonologically-Weighted Levenstein Distance (PWLD)

def PWLD(s, t):
        ''' From Wikipedia article; Iterative with two matrix rows. '''
        if s == t: return 0
        elif len(s) == 0: return len(t)
        elif len(t) == 0: return len(s)
        v0 = [None] * (len(t) + 1)
        v1 = [None] * (len(t) + 1)
        for i in range(len(v0)):
            v0[i] = i
        for i in range(len(s)):
            v1[0] = i + 0.5
            for j in range(len(t)):
                cost = 0 if s[i] == t[j] else phone2dist[s[i]][t[j]]
                v1[j + 1] = min(v1[j] + 0.5, v0[j + 1] + 0.5, v0[j] + cost)
            for j in range(len(v0)):
                
                
                v0[j] = v1[j]
                
        return v1[len(t)]

In [101]:
word2ipa = {}

word2ipa['sicher'] = 'z ɪ ç ɐ'.split()
word2ipa['Becher'] = 'b ɛ ç ɐ'.split()
#word2ipa['Becher'] = 'b ɛ ç ɛ'.split()
word2ipa['Fischer'] = 'f ɪ ʃ ɐ'.split()
word2ipa['Lichter'] = 'l ɪ ç t ɐ'.split()
word2ipa['sitzt'] = 'z ɪ t s t'.split()
word2ipa['sitzen'] = 'z ɪ t s ə n'.split()
word2ipa['tzt'] = 't s t'.split()
word2ipa['cher'] = 'ç ɐ'.split()
word2ipa['t'] = 't'.split()
word2ipa['er'] = 'ɐ'.split()

for w in word2ipa.keys():
    print('sicher', w, PWLD(word2ipa['sicher'], word2ipa[w]), LD(word2ipa['sicher'], word2ipa[w]))

sicher sicher 0 0
sicher Becher 0.2631578947368421 2
sicher Fischer 0.3684210526315789 2
sicher Lichter 0.631578947368421 2
sicher sitzt 1.3421052631578947 3
sicher sitzen 1.3421052631578947 4
sicher tzt 1.3947368421052633 4
sicher cher 1.236842105263158 2
sicher t 1.6052631578947367 4
sicher er 1.6842105263157894 3


In [90]:
 PWLD(word2ipa['tzt'], word2ipa['cher'])

0.7947194142390263

In [92]:
 PWLD(word2ipa['t'], word2ipa['er'])

0.7947194142390263

In [93]:
phone2dist['t']['ɐ'] 

0.631578947368421

In [96]:
phone2dist['ts']['ç']

0.0

In [97]:
phone2dist.keys()

dict_keys(['yː', 's', 'ɛ', 'd', 'ŋ', 'uː', 'f', 'ɛː', 'b', 'eː', 'r', 'ʃ', 'a', 'iː', 'ɐ', 'h', 'ʀ', 'oː', 'œ', 'ɡ', 'm', 'z', 'v', 'x', 'ʏ', 'ə', 'ɪ', 'aː', 'k', 'ç', 'øː', 'ʊ', 'n', 'ɔ', 'i', 't', 'ʒ', 'j', 'l', 'p', 'ts'])

In [72]:
word2ipa['sitzt']

['z', 'ɪ', 't', 's', 't']

In [12]:
PWLD('k ɪ ʀ ç ə'.split(), 'k ɪ ʀ ʃ ə'.split())

0.15789473684210525

In [13]:
LD('k ɪ ʀ ç ə'.split(), 'k ɪ ʀ ʃ ə'.split())

1

In [14]:
PWLD(word2ipa['Lösung'].split(), word2ipa['Lösung'].split())

0

In [15]:
PWLD(word2ipa['Lösung'].split(), word2ipa['schließen'].split())

1.2105263157894737

In [16]:
PWLD(word2ipa['schlossen'].split(), word2ipa['schließen'].split())

0.21052631578947367

In [17]:
word2ipa['abschließen']

'a p ʃ l iː s ə n'

In [18]:
word2ipa['schließen']

'ʃ l iː s ə n'